# README.

This notebook is the entrypoint for Azure ML enabled training.
In its essence, it connects to Azure ML, makes sure that everything is ready over there, and starts the training.
To that end, this notebook gathers all necessary sourcecodes in a temp-folder, which will be pushed to Azure ML for training.

# Imports.

In [20]:
from azureml.core import Workspace, Dataset
from azureml.core import Experiment
import os
import shutil

# Create temp folder and copy code.

Here you have to be very precise, which code to copy.
And most importantly, which code NOT to copy.

In [28]:
print("Creating temp folder...")
temp_path = "tmp_train"
if os.path.exists(temp_path):
    shutil.rmtree(temp_path)
os.mkdir(temp_path)

print("Copying files...")
shutil.copy(os.path.join("code", "train.py"), temp_path)
shutil.copy(os.path.join("code", "preprocessing.py"), temp_path)

print("Done.")

Creating temp folder...
Copying files...
Done.


# Connect to azure workspace.

Make sure that you have a config.json file with the keys subscription_id, resource_group, and cgm-ml-dev. Either here (not so nice) or in a parent folder (okay but not perfect), or in the root folder of this repo (way to go).

In [22]:
#workspace = Workspace.from_config()
#workspace
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '9b82ecea-6780-4b85-8acf-d27d79028f07'
resource_group = 'cgm-ml-dev'
workspace_name = 'cgm-ml-dev'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='depth_npy_allchannel_nonanon')
dataset.download(target_path='.', overwrite=False)

RuntimeError: File "/data/home/cpfitzner/notebooks/cgm-ml-service/Models/CNNDepthMap/CNNDepthMap-height/s5-cnndepthmap-height/MH_WHH_0026/pcrgb_MH_WHH_0026_1536918291548_104_000.p" already exists. Set overwrite=True to overwrite it.

# Get the experiment.

- You should always arrange all your runs in an experiment.
- Create at least one experiment per sprint.
- Make sure that the name of the experiment reflects the sprint number.
- On top of that you could also add other tokens to the name. For example network architecture, dataset name, and/or targets.

In [23]:
from azureml.core import Experiment
experiment_name = "s5-cnndepthmap-weight"
experiment = Experiment(workspace=workspace, name=experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
s5-cnndepthmap-weight,cgm-ml-dev,Link to Azure Machine Learning studio,Link to Documentation


# Find/create a compute target.

Connects to a compute cluster on Azure ML.
If the compute cluster does not exist, it will be created.

Note: Usually computer clusters autoscale. This means that new nodes are created when necessary. And unused VMs will be shut down.

In [24]:
import os
import urllib
import shutil
import azureml
 
from azureml.core import Experiment
from azureml.core import Workspace, Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "tristan-gpu"

# Compute cluster exists. Just connect to it.
try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print("Found existing compute target.")

# Compute cluster does not exist. Create one.    
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_NC6', 
        max_nodes=4
    )
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
compute_target

Found existing compute target.


AmlCompute(workspace=Workspace.create(name='cgm-ml-dev', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-dev'), name=tristan-gpu, id=/subscriptions/9b82ecea-6780-4b85-8acf-d27d79028f07/resourceGroups/cgm-ml-dev/providers/Microsoft.MachineLearningServices/workspaces/cgm-ml-dev/computes/tristan-gpu, type=AmlCompute, provisioning_state=Succeeded, location=westeurope, tags=None)

# Get the dataset for training.

Here you specify which dataset to use.

Note: Double check on Azure ML that you are using the right one.

In [25]:
#dataset_name = "Depth_npy"
#dataset = workspace.datasets[dataset_name]
dataset

{
  "source": [
    "('premiumfileshare', '2020_03_11_21_56_18-depth_npy_all_channels_azure_ml/npy/**')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "f0aee6b0-b462-4fb6-b18a-25062d660b95",
    "name": "depth_npy_allchannel_nonanon",
    "version": 1,
    "description": "This dataset contains all channels and is based on non-anon data. ",
    "workspace": "Workspace.create(name='cgm-ml-dev', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-dev')"
  }
}

# Push the training source code to Azure.

Creates an estimator (a template for a compute cluster node) and pushes it to the compute cluster.

In [26]:
from azureml.core.runconfig import MpiConfiguration
from azureml.train.dnn import TensorFlow
TensorFlow.get_supported_versions()

['1.10', '1.12', '1.13', '2.0']

In [29]:
# Specify pip packages here.
pip_packages = [
    "azureml-dataprep[fuse,pandas]",
    "glob2",
    #"tensorflow==2.1.0"
]

# Create the estimator.
estimator = TensorFlow(
    source_directory=temp_path,
    compute_target=compute_target,
    entry_script="train.py",
    use_gpu=True,
    framework_version="2.0",
    inputs=[dataset.as_named_input("dataset").as_mount()],
    pip_packages=pip_packages
)

# Set compute target.
estimator.run_config.target = compute_target

# Run the experiment.
run = experiment.submit(estimator)

# Show outputs.
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

# Delete temp folder.

After all code has been pushed to Azure ML, the temp folder will be removed.

In [ ]:
shutil.rmtree(temp_path)